In [5]:
import numpy as np
import pandas as pd
from math import exp

def r(t):
    """Instantaneous rate function r(t)."""
    return 0.05 / (1 + 2 * np.exp(-(1 + t)**2))

def simpsons_rule(f, a, b, n):
    """Approximate the integral of f from a to b using Simpson's Rule with n intervals (n must be even)."""
    if n % 2 != 0:
        raise ValueError("n must be even")
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return h / 3 * (fx[0] + 2 * sum(fx[2:n:2]) + 4 * sum(fx[1:n:2]) + fx[n])

def discount_factors_convergence(t_values, n_values):
    """Compute discount factors at given t_values for each n in n_values."""
    data = {t: [] for t in t_values}
    for n in n_values:
        for t in t_values:
            integral = simpsons_rule(r, 0, t, n)
            discount = exp(-integral)
            data[t].append(round(discount, 8))
    df_conv = pd.DataFrame(data, index=n_values)
    df_conv.index.name = "n"
    return df_conv

# Define time points and n values to test
t_values = [0.5, 1.0, 1.5]
n_values = [2**i for i in range(2, 8)]  # n = 4, 8, 16, ..., 128

# Generate table
df_convergence = discount_factors_convergence(t_values, n_values)


In [8]:
print(df_convergence.to_latex(float_format="%.6f"))

\begin{tabular}{lrrr}
\toprule
 & 0.500000 & 1.000000 & 1.500000 \\
n &  &  &  \\
\midrule
4 & 0.982569 & 0.960514 & 0.937138 \\
8 & 0.982569 & 0.960516 & 0.937148 \\
16 & 0.982569 & 0.960516 & 0.937148 \\
32 & 0.982569 & 0.960516 & 0.937148 \\
64 & 0.982569 & 0.960516 & 0.937148 \\
128 & 0.982569 & 0.960516 & 0.937148 \\
\bottomrule
\end{tabular}



In [1]:
import numpy as np
import pandas as pd

# Define the instantaneous rate function r(t)
def r(t):
    return 0.05 / (1 + 2 * np.exp(-(1 + t)**2))

# Simpson's Rule
def simpson(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Compute discount factor with adaptive Simpson's rule
def compute_discount_factor(t, tol=1e-6):
    n = 2
    I_old = simpson(r, 0, t, n)
    disc = np.exp(-I_old)
    logs = [(n, disc)]

    while True:
        n *= 2
        I_new = simpson(r, 0, t, n)
        disc_new = np.exp(-I_new)
        logs.append((n, disc_new))
        if abs(disc_new - disc) < tol:
            break
        disc = disc_new

    return pd.DataFrame(logs, columns=["n", f"D({t})"])

# Times to evaluate (in years)
times = [0.5, 1.0, 1.5, 2.0]
df_list = [compute_discount_factor(t) for t in times]

# Merge into one DataFrame
merged = df_list[0]
for df in df_list[1:]:
    merged = pd.merge(merged, df, on="n", how="outer")

merged = merged.sort_values("n").reset_index(drop=True)

# Format output for 6 decimals (except 2y with 8)
for col in merged.columns[1:]:
    precision = 8 if col == "D(2.0)" else 6

merged


,n,D(0.5),D(1.0),D(1.5),D(2.0)
0,2,0.982568,0.960434,0.936715,0.913405
1,4,0.982569,0.960514,0.937138,0.913981
2,8,NaN,0.960516,0.937148,0.914040
3,16,NaN,0.960516,0.937148,0.914041
4,32,NaN,NaN,NaN,0.914041


In [16]:
# evaluate 2.5*D(0.5) + 2.5*D(1) + 2.5*D(1.5) + 102.5*D(2.0) with the highest percision from the table
result = 2.5 * merged["D(0.5)"].dropna().iloc[-1] + 2.5 * merged["D(1.0)"].dropna().iloc[-1] + \
         2.5 * merged["D(1.5)"].dropna().iloc[-1] + 2.5 * merged["D(2.0)"].dropna().iloc[-1]

In [ ]:
result

9.485686430258085

In [18]:
100 * merged["D(2.0)"].dropna().iloc[-1]

91.40413111643645

In [19]:
2.5 * merged["D(0.5)"].dropna().iloc[-1] + 2.5 * merged["D(1.0)"].dropna().iloc[-1] + \
         2.5 * merged["D(1.5)"].dropna().iloc[-1] + 102.5 * merged["D(2.0)"].dropna().iloc[-1]

100.88981754669454

In [6]:
print(merged.to_latex(float_format="%.8f", index=False))

\begin{tabular}{rrrrr}
\toprule
n & D(0.5) & D(1.0) & D(1.5) & D(2.0) \\
\midrule
2 & 0.98256846 & 0.96043413 & 0.93671482 & 0.91340548 \\
4 & 0.98256855 & 0.96051378 & 0.93713849 & 0.91398132 \\
8 & NaN & 0.96051612 & 0.93714794 & 0.91403997 \\
16 & NaN & 0.96051626 & 0.93714845 & 0.91404123 \\
32 & NaN & NaN & NaN & 0.91404131 \\
\bottomrule
\end{tabular}



In [10]:
import numpy as np
import pandas as pd

# Updated Simpson's rule (user's version)
def simpson(f, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    fx = f(x)
    return h / 3 * (fx[0] + 2 * np.sum(fx[2:n:2]) + 4 * np.sum(fx[1:n:2]) + fx[n])

# Adaptive Simpson's rule with tolerance checked on N(t), not the integral
def adaptive_simpson_on_N(f, a, b, tol):
    results = []
    n = 4
    prev_integral = simpson(f, a, b, n)
    prev_N = 0.5 + prev_integral / np.sqrt(2 * np.pi)
    results.append((n, prev_N))
    while True:
        n *= 2
        curr_integral = simpson(f, a, b, n)
        curr_N = 0.5 + curr_integral / np.sqrt(2 * np.pi)
        results.append((n, curr_N))
        if abs(curr_N - prev_N) < tol:
            break
        prev_N = curr_N
    return results

def compute_N_t(t, tol=1e-12):
    f = lambda x: np.exp(-x**2 / 2)
    if t == 0:
        return [(4, 0.5)]
    elif t > 0:
        return adaptive_simpson_on_N(f, 0, t, tol)
    else:
        return adaptive_simpson_on_N(f, t, 0, tol)

# Compute for t = 0.1, 0.5, 1.0 with correct tolerance checking
results_01 = compute_N_t(0.1)
results_05 = compute_N_t(0.5)
results_10 = compute_N_t(1.0)

# Combine results into a DataFrame with NaN fill for unequal lengths
df = pd.DataFrame({
    "n_0.1": pd.Series([v for _, v in results_01], index=[n for n, _ in results_01]),
    "n_0.5": pd.Series([v for _, v in results_05], index=[n for n, _ in results_05]),
    "n_1.0": pd.Series([v for _, v in results_10], index=[n for n, _ in results_10]),
})


In [11]:
df

,n_0.1,n_0.5,n_1.0
4,0.539828,0.691463,0.841355
8,0.539828,0.691463,0.841345
16,0.539828,0.691462,0.841345
32,0.539828,0.691462,0.841345
64,NaN,0.691462,0.841345
128,NaN,0.691462,0.841345
256,NaN,0.691462,0.841345
512,NaN,NaN,0.841345
